In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import wandb
import os
import torch.optim as optimizers

In [3]:
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import pickle
import torch
import torch.nn as nn
import tqdm
import copy
import pandas as pd
#torch.multiprocessing.set_sharing_strategy('file_system') # this is important on local machine
#def set_worker_sharing_strategy(worker_id: int) -> None:
#    torch.multiprocessing.set_sharing_strategy('file_system')

In [4]:
import sys
sys.path = ['../../../src'] + sys.path
from dfs_transformer import EarlyStopping, DFSCodeSeq2SeqFC, smiles2graph

Using backend: pytorch
2021-09-10 11:44:34.267237: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt/lib:/opt/intel/tbb/lib:/opt/intel/clck/2019.0/lib:/opt/intel/compilers_and_libraries_2019/linux/lib:/opt/intel/compilers_and_libraries/linux/lib:/opt/intel/itac/2019.0.018/lib:/opt/intel/itac_2019/intel64/lib:/opt/intel/itac_latest/intel64/lib:/opt/intel/parallel_studio_xe_2019.0.045/clck_2019/lib:/opt/intel/parallel_studio_xe

In [5]:
wandb.init(project='pubchem', entity='chrisxx', name='bert10k')

config = wandb.config
config.mode = "min2min" #rnd2rnd
config.fraction_missing = 0.25
config.n_atoms = 118
config.n_bonds = 5
config.emb_dim = 120
config.nhead = 12
config.nlayers = 6
config.max_nodes = 250
config.max_edges = 500
config.dim_feedforward = 2048
config.n_files = 10
config.n_splits = 1
config.n_iter_per_split = 10000
config.lr = 0.0003
config.n_epochs = 10000
config.lr_adjustment_period = 500
config.patience = 5
config.factor = 0.96
config.minimal_lr = 6e-8
config.batch_size = 50
config.accumulate_grads = 4
config.valid_patience = 10000
config.valid_minimal_improvement=0.00
config.model_dir = "../../../models/pubchem_mini/features_selfattention/medium/"
#config.data_dir = "/mnt/project/pubchem_noH/"
config.data_dir = "../../../results/pubchem/mini10k/"
config.pretrained_dir = None#"../../models/chembl/better_transformer/medium/"
config.num_workers = 0
config.prefetch_factor = 2
config.persistent_workers = False
config.load_last = False


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-09-10 11:44:36.939318: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt

In [6]:
os.makedirs(config.model_dir, exist_ok=True)

In [7]:
path = config.data_dir

In [8]:
class PubChem(Dataset):
    """PubChem dataset of molecules and minimal DFS codes."""
    def __init__(self, path=path, n_used = 8, n_splits = 64, max_nodes=config.max_nodes,
                 max_edges=config.max_edges, useHs=False, addLoops=False, memoryEfficient=False,
                 transform=None, n_mols_per_dataset=np.inf, node_feature_key='x', edge_feature_key='edge_attr'):
        self.path = path
        self.data = []
        self.path = path
        self.n_used = n_used
        self.n_splits = n_splits
        self.useHs = useHs
        self.addLoops = addLoops
        self.memoryEfficient = memoryEfficient
        self.n_mols_per_dataset = n_mols_per_dataset
        self.max_nodes = max_nodes
        self.max_edges = max_edges
        self.node_feature_key = node_feature_key
        self.edge_feature_key = edge_feature_key
        self.prepare()
        
        
    def prepare(self):
        codes_all = {}
        d_all = {}
        i2didx = {}
        perm = np.random.permutation(self.n_splits)
        for i in tqdm.tqdm(perm[:self.n_used]):
            dname = glob.glob(self.path+"/%d/min_dfs_codes_split*.json"%(i+1))[0]
            didx = int(dname.split("split")[-1][:-5])
            dname2 = self.path+"/%d/data_split%d.json"%(i+1, didx)
            with open(dname, 'r') as f:
                codes = json.load(f)
                for key, val in codes.items():
                    codes_all[key] = val
            with open(dname2, 'r') as f:
                d_dict = json.load(f)
                for key, val in d_dict.items():
                    d_all[key] = val
        
        for smiles, code in tqdm.tqdm(codes_all.items()):
            if len(self.data) > self.n_mols_per_dataset:
                break
            if code['min_dfs_code'] is not None and len(code['min_dfs_code']) > 1:
                d = d_all[smiles]
                #d = smiles2graph(smiles, useHs=self.useHs, addLoops=self.addLoops, skipCliqueCheck=False)
                if len(d['z']) > self.max_nodes:
                    continue
                if len(d['edge_attr']) > 2*self.max_edges:
                    continue
                
                
                z = torch.tensor(d['z'], dtype=torch.long)
                x = torch.tensor(d['x'], dtype=torch.float32)
                
                
                data_ = Data(x=x,
                             z=z,
                             edge_attr=torch.tensor(d['edge_attr']),
                             edge_index=torch.tensor(d['edge_index'], dtype=torch.long),
                             min_dfs_code=torch.tensor(code['min_dfs_code']),
                             min_dfs_index=torch.tensor(code['dfs_index'], dtype=torch.long),
                             smiles=smiles,
                             node_features=torch.tensor(d['atom_features'], dtype=torch.float32),
                             edge_features=torch.tensor(d['bond_features'], dtype=torch.float32))
                self.data += [data_]   
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [9]:
def collate_fn_features(dlist):
    node_batch = [] 
    edge_batch = []
    rnd_code_batch = []
    min_code_batch = []
    for d in dlist:
        rnd_code, rnd_index = dfs_code.rnd_dfs_code_from_torch_geometric(d, 
                                                                         d.z.numpy().tolist(), 
                                                                         np.argmax(d.edge_attr.numpy(), axis=1))
        node_batch += [d.node_features]
        edge_batch += [d.edge_features]
        rnd_code_batch += [torch.tensor(rnd_code)]
        min_code_batch += [d.min_dfs_code]
    return rnd_code_batch, node_batch, edge_batch, min_code_batch

In [10]:
ngpu=1
device = torch.device('cuda:0' if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

In [11]:
to_cuda = lambda T: [t.to(device) for t in T]

In [12]:
dataset = PubChem(n_used = 1, n_splits=config.n_files)
loader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True, pin_memory=False, collate_fn=collate_fn_features,
                   num_workers=config.num_workers, prefetch_factor=config.prefetch_factor, 
                    persistent_workers=config.persistent_workers)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 997/997 [00:00<00:00, 8307.52it/s]


In [13]:
data = next(iter(loader))

In [14]:
n_node_features = data[1][0].shape[1]
n_edge_features = data[2][0].shape[1]

In [15]:
print(n_node_features, n_edge_features)

133 14


In [16]:
codes = data[-1]

In [17]:
codes

[tensor([[ 0,  1,  6,  0,  6,  3,  6,  1],
         [ 1,  2,  6,  0,  6,  1,  4,  4],
         [ 2,  3,  6,  0,  6,  4,  8,  5],
         [ 3,  4,  6,  0,  7,  5, 12,  7],
         [ 4,  5,  7,  0,  6,  7, 16, 15],
         [ 5,  6,  6,  0,  6, 15, 33, 16],
         [ 6,  7,  6,  0,  7, 16, 35, 17],
         [ 7,  2,  7,  0,  6, 17, 36,  4],
         [ 7,  8,  7,  0,  6, 17, 38, 18],
         [ 8,  9,  6,  0,  8, 18, 41, 20],
         [ 8, 10,  6,  1,  8, 18, 40, 19],
         [ 4, 11,  7,  0,  6,  7, 15,  8],
         [11, 12,  6,  2,  6,  8, 18,  9],
         [12, 13,  6,  2,  6,  9, 21, 10],
         [13, 14,  6,  2,  6, 10, 23, 11],
         [14, 15,  6,  0,  8, 11, 25, 12],
         [14, 16,  6,  2,  6, 11, 26, 13],
         [16, 17,  6,  2,  6, 13, 29, 14],
         [17, 11,  6,  2,  6, 14, 30,  8],
         [ 3, 18,  6,  1,  8,  5, 11,  6],
         [ 1, 19,  6,  0,  6,  1,  1,  0],
         [ 1, 20,  6,  0,  6,  1,  2,  2]]),
 tensor([[ 0,  1,  6,  0,  6,  6, 13,  7],
         

In [18]:
def BERTize(codes):
    inputs = []
    targets = []
    for code in codes:
        n = len(code)
        perm = np.random.permutation(n)
        target_idx = perm[:int(config.fraction_missing*n)]
        input_idx = perm[int(config.fraction_missing*n):]
        inp = code.clone()
        target = code.clone()
        target[input_idx] = -1
        inp[target_idx] = -1
        inputs += [inp]
        targets += [target]
    return inputs, targets

In [19]:
inputs, targets = BERTize(codes)

In [20]:
inputs

[tensor([[ 0,  1,  6,  0,  6,  3,  6,  1],
         [ 1,  2,  6,  0,  6,  1,  4,  4],
         [ 2,  3,  6,  0,  6,  4,  8,  5],
         [ 3,  4,  6,  0,  7,  5, 12,  7],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [ 5,  6,  6,  0,  6, 15, 33, 16],
         [ 6,  7,  6,  0,  7, 16, 35, 17],
         [ 7,  2,  7,  0,  6, 17, 36,  4],
         [ 7,  8,  7,  0,  6, 17, 38, 18],
         [ 8,  9,  6,  0,  8, 18, 41, 20],
         [ 8, 10,  6,  1,  8, 18, 40, 19],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [12, 13,  6,  2,  6,  9, 21, 10],
         [13, 14,  6,  2,  6, 10, 23, 11],
         [14, 15,  6,  0,  8, 11, 25, 12],
         [14, 16,  6,  2,  6, 11, 26, 13],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [ 3, 18,  6,  1,  8,  5, 11,  6],
         [ 1, 19,  6,  0,  6,  1,  1,  0],
         [ 1, 20,  6,  0,  6,  1,  2,  2]]),
 tensor([[ 0,  1,  6,  0,  6,  6, 13,  7],
         

In [21]:
targets

[tensor([[-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [ 4,  5,  7,  0,  6,  7, 16, 15],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [ 4, 11,  7,  0,  6,  7, 15,  8],
         [11, 12,  6,  2,  6,  8, 18,  9],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [16, 17,  6,  2,  6, 13, 29, 14],
         [17, 11,  6,  2,  6, 14, 30,  8],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1]]),
 tensor([[-1, -1, -1, -1, -1, -1, -1, -1],
         

In [22]:
model = DFSCodeSeq2SeqFC(n_atoms=config.n_atoms,
                         n_bonds=config.n_bonds, 
                         emb_dim=config.emb_dim, 
                         nhead=config.nhead, 
                         nlayers=config.nlayers, 
                         max_nodes=config.max_nodes, 
                         max_edges=config.max_edges,
                         atom_encoder=nn.Linear(n_node_features, config.emb_dim), 
                         bond_encoder=nn.Linear(n_edge_features, config.emb_dim))

In [23]:
if config.pretrained_dir is not None:
    model.load_state_dict(torch.load(config.pretrained_dir+'checkpoint.pt'))

In [24]:
if config.load_last:
    model.load_state_dict(torch.load(config.model_dir+'checkpoint.pt'))

In [25]:
optim = optimizers.Adam(model.parameters(), lr=config.lr)

lr_scheduler = optimizers.lr_scheduler.ReduceLROnPlateau(optim, mode='min', verbose=True, patience=config.patience, factor=config.factor)
#lr_scheduler = optimizers.lr_scheduler.ExponentialLR(optim, gamma=config.factor)

early_stopping = EarlyStopping(patience=config.valid_patience, delta=config.valid_minimal_improvement,
                              path=config.model_dir+'checkpoint.pt')
bce = torch.nn.BCEWithLogitsLoss()
ce = torch.nn.CrossEntropyLoss(ignore_index=-1)
softmax = nn.Softmax(dim=2)

In [26]:
model.to(device)

DFSCodeSeq2SeqFC(
  (encoder): DFSCodeEncoder(
    (emb_dfs): PositionalEncoding(
      (dropout): Dropout(p=0, inplace=False)
    )
    (emb_seq): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (emb_atom): Linear(in_features=133, out_features=120, bias=True)
    (emb_bond): Linear(in_features=14, out_features=120, bias=True)
    (mixer): Linear(in_features=600, out_features=600, bias=True)
    (enc): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=600, out_features=600, bias=True)
          )
          (linear1): Linear(in_features=600, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=600, bias=True)
          (norm1): LayerNorm((600,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((600,), eps=1e-05, 

In [ ]:
try:
    for epoch in range(config.n_epochs):
        epoch_loss = 0
        for split in range(config.n_splits):
            
            n_ids = config.n_files//config.n_splits
            dataset = PubChem(n_used = n_ids, n_splits = config.n_files)
            loader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True, pin_memory=False, 
                                collate_fn=collate_fn_features,
                   num_workers=config.num_workers, prefetch_factor=config.prefetch_factor, 
                    persistent_workers=config.persistent_workers)
            for j in range(config.n_iter_per_split):
                pbar = tqdm.tqdm(loader)
                for i, data in enumerate(pbar):
                    if i % config.accumulate_grads == 0: #bei 0 wollen wir das
                        optim.zero_grad()
                    rndc, nattr, eattr, minc = data
                    if config.mode == "min2min":
                        code = to_cuda(minc)
                    elif config.mode == "rnd2rnd":
                        code = to_cuda(rndc)
                    else:
                        raise ValueError("unrecognized config.mode")
                    nattr = to_cuda(nattr)
                    eattr = to_cuda(eattr)
                    #prepare inputs
                    inputs, targets = BERTize(code)
                    #prepare labels
                    targetc = [torch.cat((c, (-1)*torch.ones((1, 8), dtype=torch.long, device=device)), dim=0) for c in targets]
                    targetc_seq = nn.utils.rnn.pad_sequence(targetc, padding_value=-1)
                    #prediction
                    dfs1, dfs2, atm1, atm2, bnd, eos = model(inputs, nattr, eattr)
                    pred_dfs1 = torch.reshape(dfs1, (-1, config.max_nodes))
                    pred_dfs2 = torch.reshape(dfs2, (-1, config.max_nodes))
                    pred_atm1 = torch.reshape(atm1, (-1, config.n_atoms))
                    pred_atm2 = torch.reshape(atm2, (-1, config.n_atoms))
                    pred_bnd = torch.reshape(bnd, (-1, config.n_bonds))
                    tgt_dfs1 = targetc_seq[:, :, 0].view(-1)
                    tgt_dfs2 = targetc_seq[:, :, 1].view(-1)
                    tgt_atm1 = targetc_seq[:, :, 2].view(-1)
                    tgt_atm2 = targetc_seq[:, :, 4].view(-1)
                    tgt_bnd = targetc_seq[:, :, 3].view(-1)
                    loss = ce(pred_dfs1, tgt_dfs1) 
                    loss += ce(pred_dfs2, tgt_dfs2)
                    loss += ce(pred_atm1, tgt_atm1)
                    loss += ce(pred_bnd, tgt_bnd)
                    loss += ce(pred_atm2, tgt_atm2)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
                    if (i+1) % config.accumulate_grads == 0:
                        optim.step() # bei 2 wollen wir das
                    epoch_loss = (epoch_loss*i + loss.item())/(i+1)
                    mask = tgt_dfs1 != -1
                    n_tgts = torch.sum(mask)
                    acc_dfs1 = (torch.argmax(pred_dfs1[mask], axis=1) == tgt_dfs1[mask]).sum()/n_tgts
                    acc_dfs2 = (torch.argmax(pred_dfs2[mask], axis=1) == tgt_dfs2[mask]).sum()/n_tgts
                    acc_atm1 = (torch.argmax(pred_atm1[mask], axis=1) == tgt_atm1[mask]).sum()/n_tgts
                    acc_atm2 = (torch.argmax(pred_atm2[mask], axis=1) == tgt_atm2[mask]).sum()/n_tgts
                    acc_bnd = (torch.argmax(pred_bnd[mask], axis=1) == tgt_bnd[mask]).sum()/n_tgts
                    curr_lr = list(optim.param_groups)[0]['lr']
                    wandb.log({'loss':epoch_loss, 'learning rate':curr_lr})
                    pbar.set_description('Epoch %d: CE %2.6f accs: %2.2f %2.2f %2.2f %2.2f %2.2f'%(epoch+1, 
                                                                                                   epoch_loss, 
                                                                                                   100*acc_dfs1,
                                                                                                   100*acc_dfs2,
                                                                                                   100*acc_atm1,
                                                                                                   100*acc_bnd,
                                                                                                   100*acc_atm2))

                    if i % config.lr_adjustment_period == 0:
                        early_stopping(epoch_loss, model)
                        lr_scheduler.step(epoch_loss)
                        if early_stopping.early_stop:
                            break

                        if curr_lr < config.minimal_lr:
                            break
                
                
            del dataset
            del loader
        

except KeyboardInterrupt:
    torch.save(model.state_dict(), config.model_dir+'_keyboardinterrupt.pt')
    print('keyboard interrupt caught')


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9982/9982 [00:00<00:00, 10907.08it/s]
Epoch 1: CE 10.274009 accs: 1.87 3.27 89.25 50.00 77.57: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [01:03<00:00,  3.14it/s]
Epoch 1: CE 9.632325 accs: 4.07 4.47 83.33 40.65 76.02: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [01:03<00:00,  3.17it/s]
Epoch 1: CE 9.429267 accs: 3.41 6.25 82.39 54.55 77.27: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [01:03<00:00,  3.16it/s]
Epoch 1: CE 9.328070 accs: 4.43 2.46 79.31 47.78 73.40: 100%|███████████████████████████████████████████████████████████████████████████████████████████

EarlyStopping counter: 1 out of 10000


Epoch 1: CE 9.236657 accs: 8.06 3.23 79.03 44.09 81.18: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [01:01<00:00,  3.23it/s]
Epoch 1: CE 9.207441 accs: 5.49 4.40 84.07 51.65 74.73: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [01:03<00:00,  3.16it/s]
Epoch 1: CE 9.192378 accs: 4.17 3.12 87.50 67.19 78.65: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [01:03<00:00,  3.17it/s]
Epoch 1: CE 9.326738 accs: 4.85 3.94 83.33 60.00 71.82:   0%|▌                                                                                                                     | 1/200 [00:00<01:16,  2.59it/s]

EarlyStopping counter: 1 out of 10000


Epoch 1: CE 9.178184 accs: 6.86 5.88 82.35 50.98 67.16: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [01:01<00:00,  3.25it/s]
Epoch 1: CE 9.247464 accs: 6.62 4.30 80.79 47.68 71.85:   0%|▌                                                                                                                     | 1/200 [00:00<01:00,  3.27it/s]

...

Epoch 1: CE 3.371602 accs: 65.05 73.30 84.95 75.24 79.13: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:58<00:00,  3.41it/s]
Epoch 1: CE 3.526894 accs: 66.67 76.24 85.15 81.19 74.59:   0%|▌                                                                                                                   | 1/200 [00:00<00:51,  3.85it/s]

EarlyStopping counter: 9 out of 10000


Epoch 1: CE 3.345084 accs: 69.90 80.10 85.71 81.63 81.12: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:57<00:00,  3.50it/s]
Epoch 1: CE 3.823534 accs: 67.31 70.33 83.24 77.47 78.57:   0%|▌                                                                                                                   | 1/200 [00:00<00:46,  4.31it/s]

EarlyStopping counter: 10 out of 10000


Epoch 1: CE 3.233194 accs: 66.94 68.06 91.11 83.61 82.78:  16%|██████████████████▍                                                                                                | 32/200 [00:08<00:56,  2.95it/s]